In [96]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier


HERE STARTS BERT

In [97]:
# from transformers import AutoTokenizer, AutoModel
import torch
# from sklearn.svm import SVC
from sentence_transformers import SentenceTransformer , util

In [98]:
# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
# model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2").to("cuda")
model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")


In [99]:
df_bert = pd.read_csv(r'D:\Projects\ATS_Model\CS_Resume.csv')

In [100]:
X_bert = df_bert['SKILLS'].tolist()
y_bert = df_bert['JOB_CATEGORY'].tolist()

In [102]:
X_embeddings = model.encode(X_bert, show_progress_bar=True, convert_to_tensor=False)

Batches: 100%|██████████| 3164/3164 [00:27<00:00, 113.11it/s]


In [ ]:
# resume_vec = model.encode(X_bert,show_progress_bar=True,convert_to_tensor=True)

Batches: 100%|██████████| 313/313 [00:02<00:00, 134.93it/s]


In [ ]:
# job_vec = model.encode(y_bert, show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 313/313 [00:02<00:00, 152.11it/s]


In [ ]:
X_embeddings = X_embeddings.cpu()


In [103]:
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(
    X_embeddings, y_bert, test_size=0.3, random_state=42
)

In [118]:

rf = RandomForestClassifier(
    n_estimators=200,    # number of trees (try 100–500)
    max_depth=None,      # let trees expand fully
    random_state=42,
    n_jobs=-1,            # use all CPU cores
    verbose=2   # <-- prints progress
)

rf.fit(X_train_bert,y_train_bert)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


building tree 15 of 200
building tree 4 of 200
building tree 2 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 11 of 200
building tree 8 of 200
building tree 10 of 200
building tree 9 of 200
building tree 12 of 200
building tree 14 of 200
building tree 16 of 200
building tree 13 of 200
building tree 1 of 200
building tree 3 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.6s


building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58 of 200
building tree 59 of 200
building tree 60 of 200
building tree 61 of 200
building tree 62 of 200
building tree 63 of 200
building tree 64 of 200
building tree 65 of 200
building tree 66 of 200
building tree 67 of 200
building tree 68

[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   42.5s


building tree 147 of 200
building tree 148 of 200
building tree 149 of 200
building tree 150 of 200
building tree 151 of 200
building tree 152 of 200
building tree 153 of 200
building tree 154 of 200
building tree 155 of 200
building tree 156 of 200
building tree 157 of 200
building tree 158 of 200
building tree 159 of 200
building tree 160 of 200
building tree 161 of 200
building tree 162 of 200
building tree 163 of 200
building tree 164 of 200
building tree 165 of 200
building tree 166 of 200
building tree 167 of 200
building tree 168 of 200
building tree 169 of 200
building tree 170 of 200
building tree 171 of 200
building tree 172 of 200
building tree 173 of 200
building tree 174 of 200
building tree 175 of 200
building tree 176 of 200
building tree 177 of 200
building tree 178 of 200
building tree 179 of 200
building tree 180 of 200
building tree 181 of 200
building tree 182 of 200
building tree 183 of 200
building tree 184 of 200
building tree 185 of 200
building tree 186 of 200


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  1.0min finished


,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [119]:
y_pred_bert = rf.predict(X_test_bert)

print("Accuracy:", accuracy_score(y_test_bert, y_pred_bert))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    0.5s


Accuracy: 0.9985510109991438


[Parallel(n_jobs=16)]: Done 200 out of 200 | elapsed:    0.8s finished


In [130]:
sample_resume = "Node.js , Express.js , react.js"
# scores = get_fit_scores(sample_resume, y_bert)
sample_vec = model.encode([sample_resume])
# predicted_job_1 = rf.predict(sample_vec)[0]
predicted_job = rf.predict(sample_vec)[0]

print("Predicted Job:", predicted_job)

Predicted Job: Full Stack Developer


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 200 out of 200 | elapsed:    0.0s finished


In [131]:
probas = rf.predict_proba(sample_vec)[0]
classes = rf.classes_

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 200 out of 200 | elapsed:    0.0s finished


In [132]:
target_job = "Full Stack Developer"

job_probs = dict(zip(classes, probas))

chance = job_probs.get(target_job, 0) * 100
print(f"Chance of {target_job}: {chance:.2f}%")
job_probs = dict(zip(classes, probas))
top3 = sorted(job_probs.items(), key=lambda x: x[1], reverse=True)[:3]

# Print nicely
print("Top 3 job matches:")
for job, prob in top3:
    print(f"{job}: {prob*100:.2f}%")


Chance of Full Stack Developer: 48.50%
Top 3 job matches:
Full Stack Developer: 48.50%
Frontend Developer: 27.00%
Backend Developer: 10.00%


In [127]:
import pickle

In [128]:
with open("rf_bert_model.pkl", "wb") as f:
    pickle.dump(rf, f)

In [129]:
import joblib
joblib.dump(model, "bert_model.pkl")

['bert_model.pkl']